<a href="https://colab.research.google.com/github/binarycode11/PapyrusTech/blob/sibgrapi/GridSearch_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install kornia
!pip install kornia_moons --no-deps
!pip install kornia_moons

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [11]:
import os
%cd /content/
folder_path = '/content/PapyrusTech'
branch = "sibgrapi"

if os.path.exists(folder_path):
    # Apaga todo o conteúdo da pasta
    !rm -rf {folder_path}/
    print(f"Todo o conteúdo da pasta {folder_path} foi apagado.")
else:
    print(f"A pasta {folder_path} não existe.")

from google.colab import userdata
token = userdata.get('token2')
# Clonar o repositório privado usando o token
!git clone -b {branch} https://{token}@github.com/binarycode11/PapyrusTech.git

%cd {folder_path}
!ls

/content
A pasta /content/PapyrusTech não existe.
Cloning into 'PapyrusTech'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 164 (delta 79), reused 74 (delta 24), pack-reused 0
Receiving objects: 100% (164/164), 20.44 MiB | 17.20 MiB/s, done.
Resolving deltas: 100% (79/79), done.
/content/PapyrusTech
Experimentos.md  global_structurer.ipynb   interfaces		      README.md		utils
experiments	 GridSearch_metrics.ipynb  LocalFeatureMatcher.ipynb  requirements.txt


In [12]:

import numpy as np
import torch
import torchvision
from kornia.utils import tensor_to_image
from typing import Dict, Tuple, List
from kornia.feature import LocalFeatureMatcher, GFTTAffNetHardNet, DescriptorMatcher,KeyNetHardNet,SIFTFeatureScaleSpace,SIFTFeature,HesAffNetHardNet
from kornia.feature import LocalFeature, LAFDescriptor, MultiResolutionDetector,ScaleSpaceDetector, SOSNet,HardNet
from kornia.feature import CornerHarris, CornerGFTT, PassLAF, LAFOrienter, LAFAffNetShapeEstimator

from utils import (
    free_memory,  # Função para liberar memória coletando objetos não utilizados e esvaziando o cache da GPU se disponível
    evaluate_matches,  # Função para avaliar correspondências entre conjuntos de referência e inspeção, retornando verdadeiros positivos, falsos positivos e falsos negativos
    set_seed,  # Função para configurar a semente e garantir a reprodutibilidade dos experimentos
    medir_tempo,  # Context Manager para medir e imprimir o tempo de execução de um bloco de código
    plot_image_with_keypoints,  # Função para plotar uma imagem e seus keypoints
    plot_tensor,  # Função para plotar um tensor PyTorch como uma imagem
    print_table,  # Função para imprimir uma matriz em formato tabular
    MyDrawMatcher  # Classe para desenhar correspondências entre imagens utilizando Kornia
)
set_seed(42)

# Importa classes e módulos do pacote 'experiments'
from experiments import (
    PreprocessPipeline,  # Classe que implementa a interface IPreprocessor para pré-processamento de imagens, incluindo normalização, redimensionamento e conversão para escala de cinza
    DelaunayGraph,  # Classe que implementa a interface IGlobalFeatureStructurer para estruturar features globais em um grafo usando triangulação Delaunay
    FloydWarshall,  # Classe que implementa a interface IGlobalMatcher para comparar similaridade global de features em grafos usando o algoritmo de Floyd-Warshall
    WoodsDataset
)

# Exemplo de uso do Context Manager para medir o tempo de execução de um bloco de código
with medir_tempo("Exemplo de Tempo"):
    # Loop para realizar um milhão de iterações como exemplo
    for i in range(1_000_000):
        pass  # Placeholder para o código a ser executado

Exemplo de Tempo: 0.04476782999995521 segundos


In [13]:

class ImageComparisonPipeline:
    # __slots__ = ['preprocessor', 'local_feature', 'descriptor_matcher', 'global_structurer', 'global_matcher']

    def __init__(self, preprocessor=None, local_feature: LocalFeature = None, descriptor_matcher: DescriptorMatcher = None, global_structurer=None, global_matcher=None):
        self.preprocessor = preprocessor
        self.local_feature = local_feature
        self.descriptor_matcher_ablation = descriptor_matcher
        self.descriptor_matcher_fixed=  DescriptorMatcher('snn', self.descriptor_matcher_ablation.th)
        self.global_structurer = global_structurer
        self.global_matcher = global_matcher


    def process_global(self, out,images,is_plot=False):
        matricesAdj = []

        for index in (0, 1):
            pts = out[f'keypoints{index}'][out['matches'][:, index]].cpu().detach().numpy()
            desc = out[f'descriptors{index}'][out['matches'][:, index]].cpu().detach().numpy()
            matrixAdj, _ = self.global_structurer(pts, desc)
            matricesAdj.append(matrixAdj)
            if is_plot:
                img = tensor_to_image(images[index].squeeze())
                DelaunayGraph.plot_delaunay(pts,_,img=img)
        return matricesAdj

    def run(self, inspection_images: torch.Tensor, reference_images: torch.Tensor,threshold=0.1, log=None,device=torch.device('cpu')) -> Dict[Tuple[int, int], float]:
        if not all([self.preprocessor, self.local_feature, self.descriptor_matcher_ablation, self.global_structurer, self.global_matcher]):
            raise ValueError("Pipeline components are not fully set.")
        n,m = inspection_images.shape[0],reference_images.shape[0]
        scores = np.zeros((n, m))
        count_match = np.zeros((n, m))
        myDraw =MyDrawMatcher()
        cache_reference = {}

        for i_index, i_image in enumerate(inspection_images):
            lafs0, responses0, descriptors0 = self.local_feature(i_image[:1][None])
            for r_index, r_image in enumerate(reference_images):
                if r_index not in cache_reference:
                    lafs1, responses1, descriptors1 = self.local_feature(r_image[:1][None])
                    cache_reference[r_index] = (lafs1, responses1, descriptors1)
                else:
                    lafs1, responses1, descriptors1 = cache_reference[r_index]

                distance_ablation, matches_ablation = self.descriptor_matcher_ablation(descriptors0[0], descriptors1[0])# ablation
                distance, matches = self.descriptor_matcher_fixed(descriptors0[0], descriptors1[0])
                out = {
                "keypoints0": lafs0[0, :, :, 2].data,#[N, 2])
                "keypoints1": lafs1[0, :, :, 2].data,#[N, 2])
                "lafs0": lafs0,#[1, N, 2, 3]
                "lafs1": lafs1,#[1, N, 2, 3]
                "descriptors0": descriptors0[0],#[N, 128])
                "descriptors1": descriptors1[0],#[N, 128])
                "matches": matches,#[M, 2])
                }


                #ignora quando existem poucos pontos, pois diminui a confianca(global)
                if(matches.shape[0]>=3):
                    try:
                        is_plot= False
                        # if  i_index==3 and r_index in [3,12]:
                        #     print("({},{})=>*".format(i_index,r_index))
                        #     is_plot= True
                        matricesAdj = self.process_global(out,images=[i_image,r_image],is_plot=is_plot)
                        score = self.global_matcher(*matricesAdj, threshold=threshold)
                    except Exception as e:
                        score = 0   # possui poucos pontos
                else:
                   score = 0

                #ignora quando existem poucos pontos, pois diminui a confianca(local)
                num_match = matches_ablation.shape[0]
                if num_match<8:
                    num_match= 0

                # if log is not None and log in ('DEBUG') and i_index == r_index and i_index%20==0:
                if log is not None and log in ('DEBUG') and ( i_index==3 and r_index in [3,12]):
                    print(i_index,r_index ,out['keypoints0'].shape,out['descriptors0'].shape,out['matches'].shape,)
                    temp = out.copy()
                    temp['matches']=matches_ablation
                    myDraw(i_image.cpu(), r_image.cpu(),temp)

                count_match[i_index, r_index] = num_match # avaliacao pelo quantidade matching
                scores[i_index, r_index] = score # avaliacao pela correspondencia local+global

        if log is not None and log in ('INFO'):
            print_table(count_match)
            print("count_match : ",evaluate_matches(count_match,8))
            print_table(scores)
            print("scores : ",evaluate_matches(scores,8))

        return count_match,scores


import torch
from torchvision import datasets, transforms
import kornia.augmentation as KA
from torch.utils.data import DataLoader, Dataset, Subset




transform_original = transforms.Compose([
    transforms.Resize((180, 180)),
    transforms.ToTensor()
])


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device',device)
someDataset = datasets.Flowers102(root='./data', split='test', download=True,transform=transform_original)
# someDataset = WoodsDataset(root='./data/woods/', train=True,transform=transform_original)
# Encontre o primeiro número par abaixo de total_size
total_size = len(someDataset)
if total_size % 2 != 0:
    total_size -= 1
indices = np.arange(total_size)
subset = Subset(someDataset, indices)

some_loader = DataLoader(subset, batch_size=100, shuffle=True,)
total_elements = len(some_loader.dataset)
total_batches = len(some_loader)
print(f'Total de elementos no dataset: {total_elements} total batch: {total_batches}')



device cpu


100%|██████████| 344862509/344862509 [00:11<00:00, 30807210.38it/s]


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


KeyboardInterrupt: 

In [ ]:
import torch
from kornia.feature import LocalFeature, LAFDescriptor, MultiResolutionDetector,SOSNet
from kornia.feature import CornerGFTT, PassLAF, LAFOrienter, LAFAffNetShapeEstimator
from kornia.feature.scale_space_detector import get_default_detector_config
# Assuming SOSNet can be correctly imported as shown before; adjust if needed.


class GFTTFeatureSosNet(LocalFeature): #0.9    |   0.005
    """Convenience module, which implements GFTT detector + SOSNet descriptor."""

    def __init__(
        self,
        num_features: int = 200,
        upright: bool = False,
        device: torch.device = torch.device("cpu"),
        config: dict = None,
    ) -> None:
        if config is None:
            config = get_default_detector_config()
        detector = MultiResolutionDetector(
            CornerGFTT(),
            num_features,
            config,
            ori_module=PassLAF() if upright else LAFOrienter(19),
            aff_module=LAFAffNetShapeEstimator(preserve_orientation=upright).eval(),  # Usa `upright` para definir `preserve_orientation`
        ).to(device)

        # Initialize your descriptor (e.g., SOSNet) as before
        # Example with SOSNet - replace with actual initialization if different
        sosnet32 = SOSNet(pretrained=True)  # Placeholder; adjust according to actual SOSNet import
        sosnet32 = sosnet32.to(device).eval()

        descriptor = LAFDescriptor(sosnet32, patch_size=32, grayscale_descriptor=True).to(device)

        super().__init__(detector, descriptor)


In [ ]:
pp = PreprocessPipeline()
delaunayG = DelaunayGraph()
floyd = FloydWarshall()

In [ ]:

import torch
from itertools import product

def grid_search_pipeline(device, dataset_loader, param_grid):
    all_results = []
    transform_inspect = KA.AugmentationSequential(
        # KA.RandomHorizontalFlip(p=0.5),
        # KA.RandomVerticalFlip(p=0.5),
        KA. RandomMedianBlur((3, 3), p = 1),
        KA.RandomPerspective(0.3, p=0.75),
        KA.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.2, p=0.75),#brightness=0.5, contrast=0.5, saturation=0.5, hue=0.15, p=0.75
        KA.RandomAffine(degrees=90, translate=(0.10, 0.10), scale=(0.90, 1.1), p=0.75),
        same_on_batch=True,
    )
    for num_features, feature_local_class, distance, threshold in product(*param_grid.values()):
        set_seed(42)
        # Configuração específica do extrator de features com a classe e número de features
        feature_extractor = feature_local_class(num_features, device=device)
        SUM_MTP,SUM_MFP,SUM_MFN=0,0,0
        SUM_STP,SUM_SFP,SUM_SFN=0,0,0
        # Configurar a pipeline com os parâmetros atuais
        pipeline = ImageComparisonPipeline(
            pp,
            feature_extractor,
            DescriptorMatcher('snn', distance),
            delaunayG,
            floyd,
        )

        # Executar a pipeline e registrar resultados para o batch atual
        with medir_tempo():
            with torch.no_grad():
                for batch_idx, (original_batch, target) in enumerate(dataset_loader):
                    original_batch = original_batch.to(device)
                    referencia_batch, remaining_batch = torch.split(original_batch, original_batch.size(0) // 2, dim=0)
                    inspection_images = transform_inspect(original_batch)

                    # print(referencia_batch.shape,inspection_images.shape)
                    matches, scores = pipeline.run(inspection_images, referencia_batch,threshold=threshold, device=device, log=None)#log='DEBUG','INFO'

                    # print_table(matches)
                    # print_table(scores)
                    MTP, MFP, MFN = evaluate_matches(matches,threshold=4)#min de 4 pontos
                    STP, SFP, SFN = evaluate_matches(scores,threshold=10)# min de 4 correspondencia (3,7)
                    SUM_MTP += MTP
                    SUM_MFP += MFP
                    SUM_MFN += MFN

                    SUM_STP += STP
                    SUM_SFP += SFP
                    SUM_SFN += SFN

                    # print((SUM_MTP,SUM_MFP,SUM_MFN),(SUM_STP,SUM_SFP,SUM_SFN))
                    free_memory()
                    # break
                result = {
                    'params': {
                        'num_features': num_features,
                        'feature_local_class': feature_local_class.__name__,
                        'distance': distance,
                        'threshold': threshold
                    },
                    'matches': (SUM_MTP,SUM_MFP,SUM_MFN),
                    'scores': (SUM_STP,SUM_SFP,SUM_SFN),
                }
                print(result)
                all_results.append(result)
    return all_results

# Grid de parâmetros
param_grid = {
    'num_features': [30],#5,10,20,30
    'feature_local': [GFTTAffNetHardNet,GFTTFeatureSosNet, KeyNetHardNet,HesAffNetHardNet,SIFTFeature, SIFTFeatureScaleSpace],#GFTTAffNetHardNet,GFTTFeatureSosNet, KeyNetHardNet,HesAffNetHardNet,SIFTFeature, SIFTFeatureScaleSpace
    'distance': [0.9],#defaul 0.8, bom 0.9 e 1.75#8,1.0,1.5
    'threshold': [0.5]#0.5,0.05
}

# Supondo que 'flowers_loader' seja o DataLoader do seu dataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device ",device)
all_results = grid_search_pipeline(device, some_loader, param_grid)
for result in all_results:
    print("Parâmetros:", result['params'])
    print("Matches:", result['matches'])
    print("Scores:", result['scores'])


device  cpu
{'params': {'num_features': 30, 'feature_local_class': 'GFTTAffNetHardNet', 'distance': 0.9, 'threshold': 0.5}, 'matches': (2701, 441, 373), 'scores': (2894, 406, 180)}
Tempo de execução: 1272.9440295010008 segundos
{'params': {'num_features': 30, 'feature_local_class': 'GFTTFeatureSosNet', 'distance': 0.9, 'threshold': 0.5}, 'matches': (2746, 346, 328), 'scores': (2908, 161, 166)}
Tempo de execução: 1163.2768958650013 segundos
{'params': {'num_features': 30, 'feature_local_class': 'KeyNetHardNet', 'distance': 0.9, 'threshold': 0.5}, 'matches': (2053, 262, 1021), 'scores': (2417, 45, 657)}
Tempo de execução: 1421.9091580679997 segundos
{'params': {'num_features': 30, 'feature_local_class': 'HesAffNetHardNet', 'distance': 0.9, 'threshold': 0.5}, 'matches': (2627, 451, 447), 'scores': (2858, 382, 216)}
Tempo de execução: 1064.6280547570022 segundos
{'params': {'num_features': 30, 'feature_local_class': 'SIFTFeature', 'distance': 0.9, 'threshold': 0.5}, 'matches': (2608, 1696,

In [ ]:
def generate_metrics_output(TP, FP, FN):
    """
    Gera uma string formatada com os valores de TP, FP, FN, Precisão (P), Recall (R) e F1-score (F1).

    Args:
    TP (int): Número de verdadeiros positivos.
    FP (int): Número de falsos positivos.
    FN (int): Número de falsos negativos.

    Returns:
    str: Uma string formatada contendo as métricas.
    """
    # Calcular Precisão, Recall e F1-score
    P = TP / (TP + FP) if TP + FP > 0 else 0  # Precisão
    R = TP / (TP + FN) if TP + FN > 0 else 0  # Recall
    F1 = 2 * (P * R) / (P + R) if P + R > 0 else 0  # F1-score

    # Formatar e retornar a string de saída
    return "[TP:{},FP:{},FN:{}]\n[P:{:.2f},R:{:.2f},F1:{:.2f}]\n".format(TP, FP, FN, P, R, F1)


from prettytable import PrettyTable
# Criando uma tabela com prettytable
table = PrettyTable()

# Adicionando cabeçalhos à tabela
table.field_names = ["Num Features", "Feature Local Class", "Distance", "Threshold", "Matches", "Scores"]

# Adicionando as linhas com os resultados
for result in all_results:
    params = result['params']
    SUM_MTP,SUM_MFP,SUM_MFN = result['matches']
    SUM_STP,SUM_SFP,SUM_SFN = result['scores']

    table.add_row([params['num_features'], params['feature_local_class'], params['distance'], params['threshold'], generate_metrics_output(SUM_MTP,SUM_MFP,SUM_MFN), generate_metrics_output(SUM_STP,SUM_SFP,SUM_SFN)])

# Imprimindo a tabela
print(table)

+--------------+-----------------------+----------+-----------+--------------------------+--------------------------+
| Num Features |  Feature Local Class  | Distance | Threshold |         Matches          |          Scores          |
+--------------+-----------------------+----------+-----------+--------------------------+--------------------------+
|      30      |   GFTTAffNetHardNet   |   0.9    |    0.5    | [TP:2701,FP:441,FN:373]  | [TP:2894,FP:406,FN:180]  |
|              |                       |          |           | [P:0.86,R:0.88,F1:0.87]  | [P:0.88,R:0.94,F1:0.91]  |
|              |                       |          |           |                          |                          |
|      30      |   GFTTFeatureSosNet   |   0.9    |    0.5    | [TP:2746,FP:346,FN:328]  | [TP:2908,FP:161,FN:166]  |
|              |                       |          |           | [P:0.89,R:0.89,F1:0.89]  | [P:0.95,R:0.95,F1:0.95]  |
|              |                       |          |     

+--------------+-----------------------+----------+-----------+-------------------------+-------------------------+
| Num Features |  Feature Local Class  | Distance | Threshold |         Matches         |          Scores         |
+--------------+-----------------------+----------+-----------+-------------------------+-------------------------+
|      30      |   GFTTAffNetHardNet   |   0.9    |    0.5    |   [TP:444,FP:56,FN:66]  |   [TP:486,FP:46,FN:24]  |
|              |                       |          |           | [P:0.89,R:0.87,F1:0.88] | [P:0.91,R:0.95,F1:0.93] |
|              |                       |          |           |                         |                         |
|      30      |   GFTTFeatureSosNet   |   0.9    |    0.5    |   [TP:451,FP:46,FN:59]  |   [TP:489,FP:27,FN:21]  |
|              |                       |          |           | [P:0.91,R:0.88,F1:0.90] | [P:0.95,R:0.96,F1:0.95] |
|              |                       |          |           |                         |                         |
|      30      |     KeyNetHardNet     |   0.9    |    0.5    |  [TP:348,FP:31,FN:162]  |   [TP:408,FP:4,FN:102]  |
|              |                       |          |           | [P:0.92,R:0.68,F1:0.78] | [P:0.99,R:0.80,F1:0.89] |
|              |                       |          |           |                         |                         |
|      30      |    HesAffNetHardNet   |   0.9    |    0.5    |  [TP:409,FP:63,FN:101]  |   [TP:469,FP:59,FN:41]  |
|              |                       |          |           | [P:0.87,R:0.80,F1:0.83] | [P:0.89,R:0.92,F1:0.90] |
|              |                       |          |           |                         |                         |
|      30      |      SIFTFeature      |   0.9    |    0.5    |  [TP:450,FP:312,FN:60]  |  [TP:496,FP:191,FN:14]  |
|              |                       |          |           | [P:0.59,R:0.88,F1:0.71] | [P:0.72,R:0.97,F1:0.83] |
|              |                       |          |           |                         |                         |
|      30      | SIFTFeatureScaleSpace |   0.9    |    0.5    |  [TP:432,FP:506,FN:78]  |  [TP:489,FP:503,FN:21]  |
|              |                       |          |           | [P:0.46,R:0.85,F1:0.60] | [P:0.49,R:0.96,F1:0.65] |
|              |                       |          |           |                         |                         |
|      30      |      SIFTFeature      |   0.8    |    0.5    |   [TP:425,FP:12,FN:85]  |   [TP:474,FP:2,FN:36]   |
|              |                       |          |           | [P:0.97,R:0.83,F1:0.90] | [P:1.00,R:0.93,F1:0.96] |
|              |                       |          |           |                         |                         |
|      30      | SIFTFeatureScaleSpace |   0.8    |    0.5    |   [TP:445,FP:38,FN:65]  |   [TP:488,FP:10,FN:22]  |
|              |                       |          |           | [P:0.92,R:0.87,F1:0.90] | [P:0.98,R:0.96,F1:0.97] |
|              |                       |          |           |                         |                         |
+--------------+-----------------------+----------+-----------+-------------------------+-------------------------+

+--------------+-----------------------+----------+-----------+-------------------------+-------------------------+
| Num Features |  Feature Local Class  | Distance | Threshold |         Matches         |          Scores         |
+--------------+-----------------------+----------+-----------+-------------------------+-------------------------+
|      30      |   GFTTAffNetHardNet   |   0.9    |    0.5    |   [TP:23,FP:3,FN:154]   |   [TP:39,FP:0,FN:138]   |
|              |                       |          |           | [P:0.88,R:0.13,F1:0.23] | [P:1.00,R:0.22,F1:0.36] |
|              |                       |          |           |                         |                         |
|      30      |   GFTTFeatureSosNet   |   0.9    |    0.5    |   [TP:29,FP:0,FN:148]   |   [TP:46,FP:1,FN:131]   |
|              |                       |          |           | [P:1.00,R:0.16,F1:0.28] | [P:0.98,R:0.26,F1:0.41] |
|              |                       |          |           |                         |                         |
|      30      |     KeyNetHardNet     |   0.9    |    0.5    |    [TP:0,FP:3,FN:177]   |    [TP:5,FP:0,FN:172]   |
|              |                       |          |           | [P:0.00,R:0.00,F1:0.00] | [P:1.00,R:0.03,F1:0.05] |
|              |                       |          |           |                         |                         |
|      30      |    HesAffNetHardNet   |   0.9    |    0.5    |   [TP:29,FP:4,FN:148]   |   [TP:43,FP:1,FN:134]   |
|              |                       |          |           | [P:0.88,R:0.16,F1:0.28] | [P:0.98,R:0.24,F1:0.39] |
|              |                       |          |           |                         |                         |
|      30      |      SIFTFeature      |   0.9    |    0.5    |   [TP:15,FP:57,FN:162]  |   [TP:44,FP:0,FN:133]   |
|              |                       |          |           | [P:0.21,R:0.08,F1:0.12] | [P:1.00,R:0.25,F1:0.40] |
|              |                       |          |           |                         |                         |
|      30      | SIFTFeatureScaleSpace |   0.9    |    0.5    |   [TP:4,FP:131,FN:173]  |    [TP:9,FP:0,FN:168]   |
|              |                       |          |           | [P:0.03,R:0.02,F1:0.03] | [P:1.00,R:0.05,F1:0.10] |
|              |                       |          |           |                         |                         |
|      30      |      SIFTFeature      |   0.8    |    0.5    |    [TP:6,FP:0,FN:171]   |   [TP:34,FP:0,FN:143]   |
|              |                       |          |           | [P:1.00,R:0.03,F1:0.07] | [P:1.00,R:0.19,F1:0.32] |
|              |                       |          |           |                         |                         |
|      30      | SIFTFeatureScaleSpace |   0.8    |    0.5    |   [TP:1,FP:14,FN:176]   |    [TP:8,FP:0,FN:169]   |
|              |                       |          |           | [P:0.07,R:0.01,F1:0.01] | [P:1.00,R:0.05,F1:0.09] |
|              |                       |          |           |                         |                         |
+--------------+-----------------------+----------+-----------+-------------------------+-------------------------+